In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[5]:


import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import time
#Q3    
    

with open('doc_links_(2019-03-18 19-02-51).txt', 'r') as f:
    data = f.readlines()


data=[]
i = 0
 
page = requests.get("https://www.yelp.com/search?find_desc=Pediatrician&find_loc=Los%20Angeles%2C%20CA") 
#https://www.yelp.com/search?find_desc=pediatrician&find_loc=New+York%2C+NY&ns=1&start=0
                      
domain = "https://www.yelp.com"

while page.status_code==200:        
    soup = BeautifulSoup(page.content, 'html.parser')
    is_captcha_on_page = soup.find("input", id="recaptcha-token") is not None
    if is_captcha_on_page:
        break
    divs = soup.findAll("div", {"class":"lemon--div__373c0__1mboc border-color--default__373c0__2oFDT"})
    if len(divs) <= 0:
        break
    for dv in divs:
        href = dv.findAll("a", {"class":"lemon--a__373c0__IEZFH link__373c0__29943 link-color--blue-dark__373c0__1mhJo link-size--inherit__373c0__2JXk5"})
        if len(href) > 0:
            href = href[0].get('href')
            review_cnt = dv.findAll("span", {"class":"lemon--span__373c0__3997G text__373c0__2pB8f reviewCount__373c0__2r4xT text-color--mid__373c0__3G312 text-align--left__373c0__2pnx_"})
            try:
                review_cnt = int(review_cnt[0].text.strip().split()[0])
            except:
                review_cnt = -1    
            if href[:8] != "/adredir" and href[:5] == "/biz/":
                data.append((domain + href, review_cnt))

    i = i+10
    page = requests.get("https://www.yelp.com/search?find_desc=Pediatrician&find_loc=Los%20Angeles%2C%20CA&start=" + str(i)) 
#https://www.yelp.com/search?find_desc=pediatrician&find_loc=New+York%2C+NY&ns=1&start=    

docset = list(set(data))
ts = time.time()
st = str(datetime.datetime.fromtimestamp(ts).strftime('(%Y-%m-%d %H-%M-%S)'))

with open('doc_links_'+st+'.txt', 'w') as f:
    for item in docset:
        f.write("%s\t" % item[0])
        f.write("%s\n" % str(item[1]))
# In[ ]:

item = "https://www.yelp.com/biz/leaf-medical-brooklyn-2?osq=pediatrician"

review_data = []
soup_data = []

for item in docset:
    page = requests.get(item[0])
    
    if page.status_code != 200:
        continue

    soup = BeautifulSoup(page.content, 'html.parser')
    if soup.find("input", id="recaptcha-token") is not None:
        continue
    
    
    soup_data.append({"item":item[0], "soup":soup})
    try:
        biz_name = " ".join([word.text.strip() for word in soup.findAll("h1", {"class": "biz-page-title embossed-text-white shortenough"})])
        biz_rating = soup.findAll("div", {"class": "i-stars"})[0]["title"]
        biz_reviews_cnt = soup.findAll("span", {"class": "review-count rating-qualifier"})[0].text.strip()
        biz_address = soup.findAll("div", {"class": "map-box-address u-space-l4"})[0].text.strip()
    except:
        pass    

    review_contents = soup.findAll("div", {"class":"review review--with-sidebar"})
    for review_content in review_contents:
        try:
            #friends_cnt = review_content.findAll("li", {"class":"friend-count responsive-small-display-inline-block"})[0].text.split()[0]
            #reviews_cnt = review_content.findAll("li", {"class":"review-count responsive-small-display-inline-block"})[0].text.split()[0]
            review_rating = review_content.findAll("div", {"class":"biz-rating biz-rating-large clearfix"})[0].find("div").find("div")["title"]
            #review_date = review_content.findAll("span", {"class":"rating-qualifier"})[0].text.strip()#["title"].split()[0]
            review_text = review_content.findAll("div", {"class":"review-content"})[0].text.strip()
            #useful = review_content.findAll("div", {}#vote-item inline-block
        
            review_data.append({"link":item[0], "soup":soup, "biz_name":biz_name, "biz_rating":biz_rating, "biz_review_cnt":biz_reviews_cnt, "biz_address":biz_address, "review_rating":review_rating, "review_text":review_text})
        except:
            pass